In [6]:
from decision_tree import Node, get_observations
import random

In [13]:
filepath = './Homework 2 data/Dbig.txt'
obs = get_observations(filepath)
# obs[:5]

train_set_size = 8192
random.seed(42)
obs_permuted = random.sample(obs, len(obs))
obs_train = obs_permuted[:train_set_size]
obs_test = obs_permuted[train_set_size:]
# obs_permuted[:5]

In [ ]:
n_values = [32, 128, 512, 2048, 8192]
answers = []

# fill answer with (decision tree, n used to train tree, number of nodes in tree, err on test set for this tree)
for n in n_values:
    obs_n = obs_train[:n]
    root = make_subtree(obs_n)
    num_nodes_in_tree = root.count_nodes()
    err_n = root.err(obs_test)
    # TODO print out for homework
    # store in answers
    pass

### 7.1 "List n, number of nodes in that tree, errn"
# TODO

### 7.2 "Plot n vs. errn. This is known as a learning curve (a single plot). "
# TODO

### 7.3 "Visualize your decision trees’ decision boundary (five plots)
# TODO
for ans in answers:
